<a href="https://colab.research.google.com/github/devhyunjun/TIL/blob/master/Treemodel_hyunjun_230520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
# # colab 나눔고딕 모듈
# !apt-get update -qq
# !apt-get install fonts-nanum* -qq
# !pip install pymysql

# 폰트 설정
font_path = '/content\fonts/NanumGothic.ttf'
import matplotlib.pyplot as plt
# font_name = plt.font_manager.fontManager.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore", category=plt.font_manager.fontManager.FontManagerWarning)

AttributeError: ignored

In [ ]:
import pymysql

#for 시각화
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

# 검증
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

# 모델import
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_room_direction_score(row):
    if row['room_direction_text'] == '남향':
        return 3
    elif row['room_direction_text'] in ['남동향', '남서향']:
        return 2
    elif row['room_direction_text'] in ['동향', '서향']:
        return 1
    else:
        return 0

def get_floor_score(row):
    if row['_floor'] == 1:
        return 0
    elif row['_floor']==2:
        return 1
    else:
        return 2

def get_ev_score(row):
    if (row['_floor'] >= 3) & (row['elevator'] == 0) :
        return 0
    else:
        return 1

def get_finaldf():
  db= pymysql.connect(

  # cursor = conn.cursor()
  df = pd.read_sql("SELECT * FROM zb_final", db, index_col='id')
  # result = cursor.fetchall()
  # for record in result:
  #     print(record)
  db.close()

  df['rent_adjusted'] = df['deposit']*0.05/12 + df['rent'] + df['manage_cost']

  # 창 반향 전환
  df['room_direction_score'] = df.apply(get_room_direction_score, axis=1)

  # 옵션 갯수, 지하철 수 숫자로 변환
  df['manage_cost_inc_num'] = df['manage_cost_inc'].str.split(',').apply(len)
  df['near_subways_num'] = df['near_subways'].str.split(',').apply(len)

  # 매물 변수 drop
  df.drop('manage_cost_inc', axis=1, inplace=True)
  df.drop('near_subways', axis=1, inplace=True)
  df.drop('options', axis=1, inplace=True)
  # 읍면동 주거유형 갯수에서 비율로
  df['tenure_self_ratio'] = df['tenure_self'] / df['tenure_total']
  df['tenure_jeonse_ratio'] = df['tenure_jeonse'] / df['tenure_total']
  df['tenure_free_ratio'] = df['tenure_free'] / df['tenure_total']
  df['tenure_monthly_ratio'] = df['tenure_monthly'] / df['tenure_total']

  # 주거유형 갯수 drop
  df.drop(['tenure_self', 'tenure_jeonse', 'tenure_free', 'tenure_monthly'], axis=1, inplace=True)

  # 층 점수화
  df['room_floor_score'] = df.apply(get_floor_score, axis=1)

  # 엘리베이터 점수화
  df['get_ev_score'] = df.apply(get_ev_score, axis=1)

  # 문자열, 코드 등 회귀변수 안쓰는 변수 drop
  drop_columns = ['address1', 'address2', '_floor' , 'room_direction_text', 'images',
          'description', 'title', 'add1', 'add2', 'add3', 'sgg_cd',
          'emd_cd_2022',
          'emd_cd_2020',
          'sido_nm',
          'sgg_nm',
          'emd_nm', 
          'building_total',
          'hhd_total'
          ]
  # 안쓰는 변수 drop 된 df 정의
  df =  df.drop(drop_columns, axis=1, inplace=False)
  # 관리비 100 이상 말이 안됨 -> 원세랑 비교
  df[df['manage_cost']>100]
  df=df.drop(df[df['manage_cost']>100].index)

  df[df['rent']>1000]
  df=df.drop(df[df['rent']>1000].index)

  df[df['size_m2']>150]
  df=df.drop(df[df['size_m2']>150].index)
  df=df.drop(df[(df['service_type']=='원룸')&(df['size_m2']>120)].index)

  # df[(df['service_type']=='원룸')&(df['size']>40)]
  df=df.drop(df[(df['service_type']=='원룸')&(df['size_m2']>99)].index)

  df=df.drop(df[(df['service_type']=='원룸')&(df['manage_cost']>50)].index)
  df = pd.get_dummies(df)

  df.drop(['deposit', 'rent','school_dist', 'elevator', 'manage_cost'], axis=1, inplace=True)
  return df



In [ ]:
df = get_finaldf()
df.head(5)
df.info()

In [ ]:
# # 훈련 데이터에 대한 MSE 계산
# mse_train = mean_squared_error(y_train, y_pred_train)
# print("Train RMSE:", np.sqrt(abs(mse_train)))

# # 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
# y_pred_test = modelLGBM.predict(X_poly_2_test)

# # 테스트 데이터에 대한 MSE 계산
# mse_test = mean_squared_error(y_test, y_pred_test)
# print("Test RMSE:", np.sqrt(abs(mse_test)))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [138]:
df_test = pd.get_dummies(df)

X = df_test.drop(['rent_adjusted'],axis=1)
y = df_test['rent_adjusted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, shuffle=True )

## 데이터분포 확인

In [ ]:
# 집값
import warnings
warnings.filterwarnings("ignore")

sns.kdeplot(df_test['rent_adjusted'])
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('rent_adjusted Kernel Density Estimation')
plt.show()

In [ ]:
# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")

sns.kdeplot(df_test['rent_adjusted'])
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('rent_adjusted Kernel Density Estimation')
plt.show()

# Tree Model

## LightGBM

### 원본데이터

In [ ]:
modelLGBM = LGBMRegressor(n_jobs=-1)

print('BASELINE -- original features')
print(len(X_train), len(X_test))
modelLGBM.fit(X_train, y_train)

y_pred_train = modelLGBM.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = modelLGBM.predict(X_test)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_pred_test, y_test)
print("Test RMSE:", np.sqrt(abs(mse_test)))
print("Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

BASELINE -- original features
25656 8552
Train RMSE: 14.376610525475263
Test RMSE: 17.599272073149464
Train, Test 차이 비율 0.22416003702428092


#### ORIGINAL DATA -  finding best parameter by using baseian method

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# Define the loss function to be optimized
def loss_function(learning_rate, num_leaves, max_depth, n_estimators, min_child_samples):
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'learning_rate': learning_rate,
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'n_estimators' : int(n_estimators),
        'min_child_samples' : int(min_child_samples)
    }
    # Train the LightGBM model with the specified parameters
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    
    # Return the negative MSE as the loss to be minimized
    return -mse

# Define the parameter ranges for the optimizer 파라미터 범위
# 일반적으로 러닝레이트가 낮고, max
pbounds = {'learning_rate': (0.01, 0.15),
           'num_leaves': (10, 50),
           'max_depth': (3, 15),
           'n_estimators' : (100, 300),
           'min_child_samples': (20,40)}

# Create the Bayesian optimizer
optimizer = BayesianOptimization(f=loss_function, pbounds=pbounds, random_state=42)

# Run the optimization
#init_point : 최적화를 진행하기 전 초기에 랜덤하게 점 뿌려주는 갯수,
# 탐색과 활용을 균형있게 고려하기 위해 초기 점의 개수를 작은 값으로 시작하고 (예: 5-10),
#  적절한 반복 횟수를 선택하는 것이 좋습니다 (예: 10-20) 조정
optimizer.maximize(init_points=5, n_iter=12)

# Get the best parameters and the best loss achieved
best_params = optimizer.max['params']
best_loss = -optimizer.max['target']

|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... | num_le... |
-------------------------------------------------------------------------------------
| 1         | -318.3    | 0.06244   | 14.41     | 34.64     | 219.7     | 16.24     |
| 2         | -435.9    | 0.03184   | 3.697     | 37.32     | 220.2     | 38.32     |
| 3         | -537.5    | 0.01288   | 14.64     | 36.65     | 142.5     | 17.27     |
| 4         | -348.9    | 0.03568   | 6.651     | 30.5      | 186.4     | 21.65     |
| 5         | -344.7    | 0.09566   | 4.674     | 25.84     | 173.3     | 28.24     |
| 6         | -313.6    | 0.15      | 15.0      | 24.71     | 206.8     | 10.0      |
| 7         | -379.8    | 0.01959   | 14.85     | 35.23     | 218.6     | 18.13     |
| 8         | -356.8    | 0.04746   | 13.7      | 26.25     | 204.7     | 11.72     |
| 9         | -336.1    | 0.06538   | 14.54     | 33.59     | 221.3     | 12.72     |
| 10        | -453.4    | 0.01581   | 14.4      | 24.9

In [ ]:
model = lgb.LGBMRegressor(**{'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 40,
 'n_estimators': 300,
 'num_leaves': 20})
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_val = model.predict(X_val)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_val, y_pred_val)
print("Test RMSE:", np.sqrt(abs(mse_test)))
print("Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

Train RMSE: 12.809444731510853
Test RMSE: 16.901648255082126
Train, Test 차이 비율 0.31946767477785937


#### 베스트 파라미터 TEST SET 검증

In [136]:
model = lgb.LGBMRegressor(**{'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 40,
 'n_estimators': 300,
 'num_leaves': 20})
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = model.predict(X_test)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_test, y_pred_test)
print("Test RMSE:", np.sqrt(abs(mse_test)))
print("Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

KeyboardInterrupt: ignored

*### vs polynomial 비교

### 로그변환

In [140]:
from sklearn.preprocessing import StandardScaler
temp_col = ['building_nonresidential_p',
 'corp_to_pop',
 'building_others_p',
 'medical_per_ppltn',
 'subway_dist',
 'cultural_venue_dist',
 'hhd_collective_p',
 'convenience_per_ppltn',
 'culture_per_ppltn',
 'hhd_private_p',
 'green_per_area',
 'ppltn_foreign_domestic_ratio',
 'convenience_store_dist',
 'hospital_dist',
 'park_per_area',
 'building_yeonlip_p',
 'grocery_per_ppltn',
 'tenure_free_ratio',
 'supermarket_dist',
 'gender_ratio',
 'gym_per_ppltn',
 'building_dandok_p',
 'shopping_per_ppltn',
 'public_institution_dist',
 'size_m2',
 'restaurant_per_ppltn',
 'ppltn_adult_p',
 'ppltn_total']

temp_X_train = X_train.copy()
temp_X_test = X_test.copy()
temp2_X_train = X_train.copy()
temp2_X_test = X_test.copy()

scaler = StandardScaler()
temp2_X_train_scaled = scaler.fit_transform(temp2_X_train)
temp2_X_test_scaled = scaler.fit_transform(temp2_X_test)
X_train_scaled = pd.DataFrame(temp2_X_train_scaled)
X_test_scaled = pd.DataFrame(temp2_X_test_scaled)


log_train= np.log(temp_X_train[temp_col])
log_test= np.log(temp_X_test[temp_col])
X_train_scaled[temp_col] = log_train
X_test_scaled[temp_col] = log_test


temp_X_train = temp_X_train[np.isfinite(X_train_scaled)]
temp_X_test = temp_X_test[np.isfinite(X_test_scaled)]

In [141]:
modelLGBM = LGBMRegressor(n_jobs=-1)

print('BASELINE -- original features')
print(len(temp_X_train), len(temp_X_test))
modelLGBM.fit(temp_X_train, y_train)

y_pred_train = modelLGBM.predict(temp_X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = modelLGBM.predict(temp_X_test)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_pred_test, y_test)
print("Test RMSE:", np.sqrt(abs(mse_test)))
print("Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

BASELINE -- original features
25656 8552
Train RMSE: 43.91372483988172
Test RMSE: 46.4663699473931
Train, Test 차이 비율 0.05812864011920719


### POLY

In [ ]:
# Remind that you are using  train, validation and test sets for analysis
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, shuffle=True )

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=True)
X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

#### Finding Best params by bayseian

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# Define the loss function to be optimized
def loss_function(learning_rate, num_leaves, max_depth, n_estimators, min_child_samples):
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'learning_rate': learning_rate,
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'n_estimators' : int(n_estimators),
        'min_child_samples' : int(min_child_samples)
    }
    # Train the LightGBM model with the specified parameters
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train_poly, y_train)
    
    # Make predictions on the validation set
    y_pred = model.predict(X_val_poly)
    
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    
    # Return the negative MSE as the loss to be minimized
    return -mse

# Define the parameter ranges for the optimizer 파라미터 범위
# 일반적으로 러닝레이트가 낮고, max
pbounds = {'learning_rate': (0.01, 0.1),
           'num_leaves': (10, 50),
           'max_depth': (3, 10),
           'n_estimators' : (100, 300),
           'min_child_samples' : (20,40)
           }

# Create the Bayesian optimizer
optimizer = BayesianOptimization(f=loss_function, pbounds=pbounds, random_state=42)

# Run the optimization
#init_point : 최적화를 진행하기 전 초기에 랜덤하게 점 뿌려주는 갯수,
# 탐색과 활용을 균형있게 고려하기 위해 초기 점의 개수를 작은 값으로 시작하고 (예: 5-10),
#  적절한 반복 횟수를 선택하는 것이 좋습니다 (예: 10-20) 조정
optimizer.maximize(init_points=5, n_iter=12)

# Get the best parameters and the best loss achieved
best_params = optimizer.max['params']
best_loss = -optimizer.max['target']

|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... | num_le... |
-------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
modelLGBM = LGBMRegressor(n_jobs=-1, min_child_samples=30)

print('BASELINE -- original features')
print(len(X_train_poly), len(X_val_poly))
modelLGBM.fit(X_train_poly, y_train)

y_pred_train = modelLGBM.predict(X_train_poly)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = modelLGBM.predict(X_val_poly)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_val, y_pred_val)
print("Test RMSE:", np.sqrt(abs(mse_test)))
print("Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

BASELINE -- polynomial features
19242 6414
Train RMSE: 12.192004916352772
Test RMSE: 17.22823689781495
Train, Test 차이 비율 0.4130766035623254


In [ ]:
best_params

#### POLY 베스트 파라미터 TEST SET 검증

In [ ]:

model = lgb.LGBMRegressor(**{'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 243,
 'num_leaves': 28})
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = model.predict(X_test)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_test, y_pred_test)
print("Test RMSE:", np.sqrt(abs(mse_test)))

KeyboardInterrupt: ignored

## GradientBoostingRegressor

### 원본데이터

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
modelGBRegressor = GradientBoostingRegressor()

print('BASELINE -- original features')
print(len(X_train), len(X_val))
modelGBRegressor.fit(X_train, y_train)

y_pred_train = modelGBRegressor.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_val = modelGBRegressor.predict(X_val)

# 테스트 데이터에 대한 MSE 계산
mse_val = mean_squared_error(y_val, y_pred_val)
print("GradientBoostingRegressor VAL RMSE:", np.sqrt(abs(mse_val)))
print("GradientBoostingRegressor Train, VAL 차이 비율", (np.sqrt(abs(mse_val))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

BASELINE -- original features
19242 6414
Train RMSE: 19.37402533090049
Test RMSE: 20.099227543729945
Train, Test 차이 비율 0.03743167464908791


### ORIGINAL DATA -  finding best parameter by using baseian method

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# Define the loss function to be optimized
def loss_function(learning_rate,n_estimators, subsample):
    params = {
        'loss': 'absolute_error',
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'subsample' : subsample
    }
    # Train the LightGBM model with the specified parameters
    model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    
    # Return the negative MSE as the loss to be minimized
    return -mse

# Define the parameter ranges for the optimizer 파라미터 범위
# 일반적으로 러닝레이트가 낮고, max
pbounds = {'learning_rate': (0.01, 0.1),
           'n_estimators' : (100, 300),
           'subsample': (0.5,1)}

# Create the Bayesian optimizer
optimizer = BayesianOptimization(f=loss_function, pbounds=pbounds, random_state=42)

# Run the optimization
#init_point : 최적화를 진행하기 전 초기에 랜덤하게 점 뿌려주는 갯수,
# 탐색과 활용을 균형있게 고려하기 위해 초기 점의 개수를 작은 값으로 시작하고 (예: 5-10),
#  적절한 반복 횟수를 선택하는 것이 좋습니다 (예: 10-20) 조정
optimizer.maximize(init_points=5, n_iter=10)

# Get the best parameters and the best loss achieved
best_params = optimizer.max['params']
best_loss = -optimizer.max['target']

|   iter    |  target   | learni... | n_esti... | subsample |
-------------------------------------------------------------
| 1         | -514.7    | 0.04371   | 290.1     | 0.866     |
| 2         | -546.4    | 0.06388   | 131.2     | 0.578     |
| 3         | -655.5    | 0.01523   | 273.2     | 0.8006    |
| 4         | -549.5    | 0.07373   | 104.1     | 0.985     |
| 5         | -529.0    | 0.08492   | 142.5     | 0.5909    |
| 6         | -485.3    | 0.08809   | 295.1     | 0.9762    |
| 7         | -504.6    | 0.09368   | 165.1     | 0.5417    |
| 8         | -924.4    | 0.01      | 183.6     | 1.0       |
| 9         | -474.8    | 0.1       | 232.8     | 0.5       |
| 10        | -515.5    | 0.05608   | 219.0     | 0.9691    |
| 11        | -819.6    | 0.01      | 247.3     | 1.0       |
| 12        | -516.1    | 0.08973   | 154.7     | 0.8899    |
| 13        | -1.072e+0 | 0.01      | 117.4     | 0.5       |
| 14        | -788.9    | 0.01195   | 226.2     | 0.5253    |
| 15    

In [ ]:
best_loss

474.7797391227841

#### 베스트 파라미터 Test SET 검증 ORIGINAL

In [ ]:
model = GradientBoostingRegressor(**{'learning_rate': 0.1, 'n_estimators': 232, 'subsample': 0.5})
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_test = model.predict(X_test)

# 테스트 데이터에 대한 MSE 계산
mse_test = mean_squared_error(y_test, y_pred_test)
print("GradientBoostingRegressor Test RMSE:", np.sqrt(abs(mse_test)))
print("GradientBoostingRegressor Train, Test 차이 비율", (np.sqrt(abs(mse_test))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

Train RMSE: 17.17284775890524
GradientBoostingRegressor Test RMSE: 19.226090791316107
GradientBoostingRegressor Train, Test 차이 비율 0.11956333982790505


#### POLYNOMIAL transformed data

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=True)
X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
modelGBRegressor = GradientBoostingRegressor()

print('BASELINE -- polynomial features')
print(len(X_train_poly), len(X_val_poly))
modelGBRegressor.fit(X_train_poly, y_train)

y_pred_train = modelGBRegressor.predict(X_train_poly)

# 훈련 데이터에 대한 MSE 계산
mse_train = mean_squared_error(y_train, y_pred_train)
print("Train RMSE:", np.sqrt(abs(mse_train)))

# 적합된 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred_val = modelGBRegressor.predict(X_val_poly)

# 테스트 데이터에 대한 MSE 계산
mse_val = mean_squared_error(y_val, y_pred_val)
print("GradientBoostingRegressor POLYNOMIAL VAL RMSE:", np.sqrt(abs(mse_val)))
print("GradientBoostingRegressor POLYNOMIAL Train, VAL 차이 비율", (np.sqrt(abs(mse_val))- np.sqrt(abs(mse_train)))/(np.sqrt(abs(mse_train))))

BASELINE -- original features
19242 6414


KeyboardInterrupt: ignored